# 一份详细的机器学习项目

在这一节中，你将通过一份详细的机器学习项目，假设你是一位最近聘请到房地产公司的数据科学家。这将是你要采取的主要步骤经过：

1. 看大图
2. 获取数据
3. 挖掘并可视化数据以获取见解
4. 为机器学习算法准备数据
5. 选择一个模型并且训练它
6. 微调你的模型
7. 提出你的解决方案
8. 启动、监控并且维护你的系统

## 使用真实数据

当您正在学习机器学习时，最好实际进行试验真实世界的数据，而不仅仅是人造数据集。 幸运的是，有成千上万的开放数据集可供选择，涵盖各种域。 这里有几个地方你可以看看获取数据：

### 流行的开源数据仓库
 - —[UC Irvine Machine Learning Repository](http://archive.ics.uci.edu/ml/index.php)
 - —[Kaggle datasets](https://www.kaggle.com/datasets)
 - —[Amazon’s AWS datasets](https://aws.amazon.com/fr/datasets/)
 
### Meta portals (他们列出了开源的数据仓库):

 - —http://dataportals.org/
 - —http://opendatamonitor.eu/.
 - —http://quandl.com/

### 其他网页列出了许多流行的开源的数据仓库：
 - —[Wikipedia’s list of Machine Learning datasets](https://goo.gl/SJHN2k)
 - —[Quora.com question](http://goo.gl/zDR78y)
 - —[Datasets subreddit](https://www.reddit.com/r/datasets)
 
 在这一节中，我们选择了在本章中，我们选择了StatLib存储库中的加州房屋价格数据集2（见图2-1）。 该数据集基于1990年加州人口普查的数据。这不完全是最近的（你可以在海湾地区买得起一间漂亮的房子时间），但它有很多学习的品质，所以我们会假装它是最近的数据。 我们还添加了分类属性并删除了一些用于教学目的的功能。
 
 <img align="left" src="figures/FIgure2-1.png">
 (图2-1)

## 看图

你被要求的第一项任务是演示使用加州人口普查在加利福尼亚州建立住房价格模型数据。 这些数据具有诸如人口，收入中位数，住房中位数等指标价格，等等在加利福尼亚州的每个块组。 块组是最小的美国人口普查局发布样本数据的地理单位（一个街区）集团通常拥有600至3000人）。 我们简称其为“地区”.


你的模型应该从这些数据中学习，并能够根据在其他地区的价格指标预测房屋的中位数。

<img align="right" src="figures/figure2-2.png">
### 构建问题

**向老板提问的第一个问题是商业目标究竟是什么？**建模可能不是最终目标。公司期望如何通过使用此模型并从中获取收益。这很重要，因为它会决定你如何构建问题，你将选择什么算法，你将使用哪种性能度量评估你的模型，以及你应该花多少精力来调整它。

你的boss回答你的模型输出结果将被送到另一台机器学习系统（见图2），还有许多其它信号。这个下游系统将决定它是否值得投资于某一地区或不是。获得这一权利至关重要，因为它直接影响收入.

```
管道

一系列数据处理组件被称为数据管道。流水线在机器学习系统中非常常见，
因为需要处理大量数据并应用许多数据转换。

组件通常异步运行。每个组件都会引入大量数据，对其进行处理，并将结
果吐出到另一个数据存储区中，然后一段时间后，管道中的下一个组件会
拉取这些数据并吐出自己的输出，等等。每个组件都是相当独立的：组件
之间的接口就是数据存储。这使得系统很容易掌握(在数据流图的帮助下),
不同的团队可以专注于不同的组件。此外，如果某个组件发生故障，那么
只需使用该组件的最后一个输出，就可以继续正常运行下游组件（至少暂
时保持一段时间）。这使得这个架构非常强大。

另一方面，如果没有实施适当的监控，一段时间内可能会忽略一个损坏的
组件。数据变得陈旧，整个系统的性能下降。

```

**接下来要问的问题是当前的解决方案是什么样的（如果有的话）**。 它会经常为您提供参考性能以及如何解决问题的见解。你的老板回答说目前地区房价是由专家手动估算的：一个团队收集有关一个区的最新信息（不包括中位数房价），他们使用复杂的规则来提出估计。 这种做法代价高昂，耗时长、估算误差大; 他们的典型错误率大约是15％。

好了，有了这些信息，您现在就可以开始设计您的系统了。``首先，您需要设定问题的框架：它是受监督的，无监督的，还是强化学习？是分类任务，回归任务还是其他？你应该使用批量问题还是在线学习技巧？``在你阅读之前，暂停并尝试自己回答这些问题。

你找到答案了吗？让我们看看：这显然是一个典型的监督式学习任务，因为您给了标注的培训示例（每个示例都带有预期输出，即该区的房价中位数）。此外，它也是一个典型的回归任务，因为要求您预测一个值。更具体地说，这是一个多变量回归问题，因为该系统将使用多个特征来进行预测（它将使用该地区的人口，收入中位数等）。在第一章中，你基于一个特征即人均GDP预测了生活满意度，所以这是一个单变量回归问题。最后，系统中没有连续的数据流，没有特别的需要快速适应数据变化，数据足够小以适应内存，所以简单的批量学习应该没问题。

#### 【如果数据非常庞大，你可以拆分批量学习跨多个服务器（稍后你会看到使用mapereduce技术）， 或者你可以使用在线学习技术来代替】

### 选一个绩效评估模式

您的下一步是选择一个绩效指标。一个典型的性能指标是均方根误差(RMSE)。它衡量标准系统在预测中出现的错误偏差。 例如， 一个$RMSE=50000$ 意味着大约有68%的预测结果会实际价格5万美元以内，大约95%的预测会在实际价格10万美元以内：

##### 计算RMSE的数学函数：$RMSE(X, h) = \sqrt{\frac{1}{m}\sum_{i=1}^m(h(x^{(i)}) - y^{(i)})^2}$

尽管RMSE通常是回归任务的首选性能指标，但在某些情况下，您可能更愿意使用其他函数。 例如，假设有很多离群区。 在这种情况下，您可以考虑使用平均绝对误差

$MAE(X, h) = \frac{1}{m}\sum_{i=1}^m|h(x^{(i)}) - y^{(i)}|$

RMSE和MAE都是测量两个向量之间距离的方法：预测向量和目标值向量。 各种距离度量或规范是可能的

### 检查假设

最后，列出并验证迄今为止（由您或其他人）做出的假设是一种很好的做法。这可以在早期发现严重问题。例如，您的系统输出的地区价格将被输入下游的机器学习系统，并且我们假设这些价格将被使用。但是，如果下游系统实际上将价格转换成类别（例如，“便宜”，“中等”或“昂贵”），然后使用这些类别而不是价格本身呢？

在这种情况下，让价格完美正确并不重要;你的系统只需要获得正确的类别。如果是这样，那么这个问题应该被定义为分类任务，而不是回归任务。在回归系统上工作数月之后，您不希望发现这一点。

幸运的是，在与负责下游系统的团队交谈之后，您确信他们确实需要实际价格，而不仅仅是类别。很好！


*首先，让我们确保 notebook 在 Python2和Python3中都能正常工作，导入一些常用模块，确保matplotlib在线绘制图形，并准备一个函数来保存图形.*

In [ ]:
from __future__ import division, print_function, unicode_literals

# common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axex.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_TO_DIR="."
CHAPTER_ID = "end_to_endproject"
IMAGES_PATH = os.path.join(PROJECT_TO_DIR, "images",CHAPTER_ID)

def save_fig(fig_id, tight_layout=True, fig_extension='png', resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id+"."+fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)